In [3]:
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
# Function to calculate distance between two lat-lng points
def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    # Distance in kilometers
    distance = R * c
    return distance

In [5]:
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
filepath = '/content/drive/My Drive/merged_rides_with_elevation.csv'

In [8]:
df = pd.read_csv(filepath)

In [1]:
df.head()

NameError: name 'df' is not defined

In [9]:
df['Start_Altitude'] = df[['Start_Altitude', 'Start_Altutude', 'State_Altitude']].bfill(axis=1).iloc[:, 0]

In [10]:
df.drop(columns=['State_Altitude', 'Start_Altutude'], inplace=True)


In [11]:
df['End_Altitude'] = df[['End_Altitude', 'end_altitude', 'end_Altitude']].bfill(axis=1).iloc[:, 0]

In [12]:
df.drop(columns=['end_Altitude', 'end_altitude'], inplace=True)

In [13]:
df['Start_Altitude'].fillna(method='ffill', inplace=True)
df['End_Altitude'].fillna(df['End_Altitude'].mean(), inplace=True)

In [14]:
from sklearn.impute import SimpleImputer
df['Elevation_Change'] = df['End_Altitude'] - df['Start_Altitude']

In [15]:
imputer = SimpleImputer(strategy='mean')
df['Elevation_Change'] = imputer.fit_transform(df[['Elevation_Change']])

In [16]:
df = df.drop(columns=['Start_Altitude', 'End_Altitude'])

In [17]:
df['Distance'] = df.apply(lambda x: haversine_distance(x['start_lat'], x['start_lng'], x['end_lat'], x['end_lng']), axis=1)

In [18]:
df = df.drop(columns=['start_lat', 'start_lng', 'end_lat', 'end_lng'])

In [19]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [20]:
df['season'] = df['started_at'].dt.month.map(get_season)

In [21]:
def get_day_of_week(timestamp):
    return timestamp.strftime("%A")

In [22]:
# Apply the function to each row in the DataFrame
df['day_of_week'] = df['started_at'].apply(get_day_of_week)

In [23]:
df['trip_duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60

In [26]:
weatherData = '/content/drive/My Drive/3667056.csv'

In [27]:
tempData = pd.read_csv(weatherData)

In [28]:
tempData['started_at'] = pd.to_datetime(tempData['DATE'])
tempData['started_at'] = tempData['started_at'].dt.date

In [29]:
df['started_at'] = df['started_at'].dt.date

In [30]:
df = pd.merge(df, tempData[['started_at', 'TMAX', 'TMIN']], on='started_at', how='left')

In [9]:
# ride_types = dfs['rideable_type'].unique()

In [10]:
# print(ride_types)

['electric_bike' 'classic_bike' 'docked_bike']


In [32]:
df['rideable_type'] = df['rideable_type'].replace({
    'electric_bike': 'electric',
    'docked_bike': 'classic',
    'classic_bike': 'classic'
})

In [14]:
# print(dfs)

                   ride_id rideable_type        started_at          ended_at  \
0         C2F7DD78E82EC875      electric   1/13/2022 11:59   1/13/2022 12:02   
1         A6CF8980A652D272      electric    1/10/2022 8:41    1/10/2022 8:46   
2         BD0F91DFF741C66D       classic    1/25/2022 4:53    1/25/2022 4:58   
3         CBB80ED419105406       classic     1/4/2022 0:18     1/4/2022 0:33   
4         DDC963BFDDA51EEA       classic    1/20/2022 1:31    1/20/2022 1:37   
...                    ...           ...               ...               ...   
11387550  F74DF9549B504A6B      electric   12/7/2023 13:15   12/7/2023 13:17   
11387551  BCDA66E761CC1029       classic   12/8/2023 18:42   12/8/2023 18:45   
11387552  D2CF330F9C266683       classic   12/5/2023 14:09   12/5/2023 14:13   
11387553  3829A0D1E00EE970      electric   12/2/2023 21:36   12/2/2023 21:53   
11387554  A373F5B447AEA508       classic  12/11/2023 13:07  12/11/2023 13:11   

                     start_station_name